<a href="https://colab.research.google.com/github/kavipriyan43/poem-generator/blob/main/Poetry_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Install and import required libraries
!pip install tensorflow numpy

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
import os

# Step 2: Upload your dataset
from google.colab import files
uploaded = files.upload()

# Replace 'your_file.txt' with the actual uploaded file name
file_name = list(uploaded.keys())[0]

# Read text
with open(file_name, 'r', encoding='utf-8') as f:
    text = f.read().lower()

print(f"Corpus length: {len(text)} characters")

# Step 3: Create character mappings
chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}
vocab_size = len(chars)
print(f"Vocabulary size: {vocab_size}")

# Step 4: Prepare sequences
seq_length = 100  # number of characters in input sequence
step = 1
sentences = []
next_chars = []

for i in range(0, len(text) - seq_length, step):
    sentences.append(text[i:i + seq_length])
    next_chars.append(text[i + seq_length])

print(f"Number of sequences: {len(sentences)}")

# One-hot encode sequences
X = np.zeros((len(sentences), seq_length, vocab_size), dtype=np.bool_)
y = np.zeros((len(sentences), vocab_size), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

# Step 5: Build the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(seq_length, vocab_size)))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

# Step 6: Define sampling function for text generation
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Step 7: Train the model with text generation callback
def on_epoch_end(epoch, logs):
    print(f"\n--- Generating text after Epoch: {epoch+1}")
    start_index = np.random.randint(0, len(text) - seq_length - 1)
    seed_text = text[start_index:start_index + seq_length]
    print(f"Seed: \"{seed_text}\"")

    generated = seed_text
    for i in range(400):  # generate 400 characters
        x_pred = np.zeros((1, seq_length, vocab_size))
        for t, char in enumerate(seed_text):
            x_pred[0, t, char_to_idx[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=0.8)
        next_char = idx_to_char[next_index]
        generated += next_char
        seed_text = seed_text[1:] + next_char
    print(generated)

from tensorflow.keras.callbacks import LambdaCallback
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Step 8: Train the model
model.fit(X, y, batch_size=128, epochs=5, callbacks=[print_callback])

# Step 9: Save the model
model.save("poetry_generator_model.h5")
print("Model saved as poetry_generator_model.h5")


Saving poems.txt to poems (2).txt
Corpus length: 73667 characters
Vocabulary size: 49
Number of sequences: 73567


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 256)            │       313,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 49)             │        12,593 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 325,937 (1.24 MB)

 Trainable params: 325,937 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 596ms/step - loss: 2.9233
--- Generating text after Epoch: 1
Seed: " soon,
getting and spending, we lay waste our powers:
little we see in nature that is ours;
we have "
 soon,
getting and spending, we lay waste our powers:
little we see in nature that is ours;
we have lnour, seand can thayt sind lo hing boke, ing feon besl,
pace her antess wan thounch the mingyeend deer be weur moy the serd wore ar oarh,
whonken
tha whee -lithe man buther bl the merd ge wrie ind sot sand livand forethtine ongr phot yand not leisi, of thild yoy shor wet end
ke nha don d ile,
whe lhy oh ah sloun.

hag
cod  hit tham at s ait arom being the whee sered ceoremf fathir ingi oure pne. 
575/575 ━━━━━━━━━━━━━━━━━━━━ 381s 660ms/step - loss: 2.9229
Epoch 2/5
575/575 ━━━━━━━━━━━━━━━━━━━━ 0s 594ms/step - loss: 2.2764
--- Generating text after Epoch: 2
Seed: " to those sounds,
and feeding thus our fancies, we advanced
along the indented shore; when suddenly,"
 to those sound

Model saved as poetry_generator_model.h5


In [8]:
# -------------------------------
# Step 10: Text Generation after training
# -------------------------------

def generate_text(model, seed_text, length=400, temperature=0.8):
    generated = seed_text
    for _ in range(length):
        x_pred = np.zeros((1, seq_length, vocab_size))
        for t, char in enumerate(seed_text):
            if char in char_to_idx:
                x_pred[0, t, char_to_idx[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx_to_char[next_index]
        generated += next_char
        seed_text = seed_text[1:] + next_char
    return generated

# Step 11: Input seed text and generate
print(f"\nEnter a seed text (at least {seq_length} characters):")
seed_text = input()
if len(seed_text) < seq_length:
    seed_text = " " * (seq_length - len(seed_text)) + seed_text

generated_text = generate_text(model, seed_text, length=400, temperature=0.5)
print("\n--- Generated Text ---\n")
print(generated_text)


Enter a seed text (at least 100 characters):
the night was silent and full of mystery, the stars shining

--- Generated Text ---

                                         the night was silent and full of mystery, the stars shining to make and soured,
and stare and the hade the rave the may wath sowe more
that shen in the mook and where all the sulling there
where whele our for wise searts.
where the plower in the blowers,
and low our hoo and to eese the rourt but and it her sunged.

a wast of the fore and may hare beer and ale the more,
and to be prowing lone though the mook
and moke to make and be in and not the soun
of m
